#Optimization Examples

This notebook is designed to run in Google Colab, compiling and executing code on the underlying virtual machine. To run the notebook, open it in Google Colab and execute each cell with "shift+enter" or by pressing the cells run button.

You can also copy and paste the code into other machines for testing.

Note, each of these examples may run different on different machines. In particular, references to static memory allocation have been removed from a prior version of this example due to the greater restrictions on stack memory in many of the virtual machines that students may encounter. The example comparing static to dynamic memory allocation has been left in its original form, but its ability to run without a segmentation fault may depend on the stack size limits and available memory on the systems being used.

In [1]:
!ulimit -s unlimited

Passing arrays to functions, or calling functions while looping through an array.

Instructor: Note to students that this is a measure of the cost of function call overhead within loops. The array size may have an impact on the difference you will see

In [2]:
%%writefile arraystride.c
#include <stdio.h>
#include <time.h>
#include <stdlib.h>

#define N 10000

int ** alloc_imatrix(int ncol, int nrow) {
    int *row;
    int **matrix; 
    int i; 
  
    matrix = (int **)malloc(sizeof(int *) * nrow + sizeof(int) * ncol * nrow); 
    row = (int *)(matrix + nrow); 
    for(i = 0; i < nrow; i++) 
        matrix[i] = (row + ncol * i); 

    return matrix;
}

double ** alloc_dmatrix(int ncol, int nrow) {
    double *row;
    double **matrix; 
    int i; 
  
    matrix = (double **)malloc(sizeof(double *) * nrow + sizeof(double) * ncol * nrow); 
    row = (double *)(matrix + nrow); 
    for(i = 0; i < nrow; i++) 
        matrix[i] = (row + ncol * i); 

    return matrix;
}



int main(int argc, char ** argv) {
    int i,j;
    int ** a = alloc_imatrix(N,N);
	int mode=-1;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: arraystride 0|1\n  0 = inner loop is j\n  1 = inner loop is i\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
        printf("EXAMPLE: Striding 2D array row-column or column-row.\n");

        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }

    start = clock();
    if(mode==0) {
		for(i=0;i<N;i+=1) {
			for(j=0;j<N;j+=1) {
				a[i][j] = i;
			}
		}
	} else if(mode==1) {
	    for(j=0;j<N;j+=1) {
			for(i=0;i<N;i+=1) {
				a[i][j] = i;
			}
		}
	} else {
        printf("ERROR: INVALID MODE ENTERED\n");
    }
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);

    free(a);
}


Writing arraystride.c


In [3]:
!gcc -O0 -o arraystride arraystride.c

In [4]:
!./arraystride 0

USAGE: arraystride 0|1
  0 = inner loop is j
  1 = inner loop is i
EXAMPLE: Striding 2D array row-column or column-row.
Running with mode 0
Elapsed Time = 0.563679


In [5]:
!./arraystride 1

USAGE: arraystride 0|1
  0 = inner loop is j
  1 = inner loop is i
EXAMPLE: Striding 2D array row-column or column-row.
Running with mode 1
Elapsed Time = 1.601648


In [6]:
%%writefile unitstride.c
#include <stdio.h>
#include <time.h>
#include <stdlib.h>
#define N 100000000

int main(int argc, char ** argv) {
    int i,j;
    int *a = (int *)malloc(sizeof(int)*N);
	int stride=1;
    int repeat=10;
    clock_t start,end;
    double elapsed_time;
    int Nstop;

    printf("USAGE: unitstride N\n  N = stridelength\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&stride);
        printf("EXAMPLE: striding through array\n");
        printf("Running with stride %d\n",stride);
    }else {
        printf("Please provide a command line argument\n");
    }

    start = clock() ;

    for(j=0;j<repeat;j+=1) 
        for(i=0;i<N;i+=stride) {
            a[i] = i;
        }
    end = clock() ;
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC ;
    
    Nstop = 0;
    for(i=0;i<N;i+=stride) Nstop++;

    printf("Time doing %d items of work with stride of %d = %lf\n",Nstop,stride,elapsed_time);



    start = clock() ;
    for(j=0;j<repeat;j+=1) 
        for(i=0;i<Nstop;i+=1) {
            a[i] = i;
        }
    end = clock() ;
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC ;

    printf("Time doing %d items of work with stride of 1 = %lf\n",Nstop,elapsed_time);

    free(a);
}



Writing unitstride.c


In [7]:
!gcc -o unitstride unitstride.c


In [8]:
!./unitstride 1000

USAGE: unitstride N
  N = stridelength
EXAMPLE: striding through array
Running with stride 1000
Time doing 100000 items of work with stride of 1000 = 0.182734
Time doing 100000 items of work with stride of 1 = 0.002162


In [9]:
%%writefile loopcondition.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

  
#define N 100000000

int main(int argc, char ** argv) {
    int i;
    int *a = (int *)malloc(sizeof(int)*N);
    int nm = N-1;
    int mode=0;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: loopcondition 0|1\n  0=no if in loop\n  1=if in loop\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
        printf("EXAMPLE: Condition inside or outside of loop\n");
        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }
    

    start = clock();
    if(mode==0) {
        // no conditions in loop
        a[0]=0;
        for(i=1;i<nm;i++) {
            a[i]=i;
        }
        a[nm]=0;
    } else {
        // conditions in loop
        for(i=0;i<N;i++) {
            if(i==0) a[i]=0;
            else if(i==N-1) a[i]=0;
            else a[i]=i;
        }
    }
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);

    free(a);
}


Writing loopcondition.c


In [10]:
!gcc -O0 -o loopcondition loopcondition.c


In [11]:
!./loopcondition 0


USAGE: loopcondition 0|1
  0=no if in loop
  1=if in loop
EXAMPLE: Condition inside or outside of loop
Running with mode 0
Elapsed Time = 0.378818


In [12]:
!./loopcondition 1

USAGE: loopcondition 0|1
  0=no if in loop
  1=if in loop
EXAMPLE: Condition inside or outside of loop
Running with mode 1
Elapsed Time = 0.475306


In [13]:
%%writefile loopinvariantcode.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

    
#define N 100000000

int main(int argc, char ** argv) {
    int i;
    int j,k;
    int *a = (int *)malloc(sizeof(int)*N);
	int mode=-1;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: loopinvariantcode 0|1\n  0=invariant in loop\n  1=invariant outside loop\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
				printf("EXAMPLE: Loop invariant code inside or outside of loop\n");
        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }

    start = clock();
	if(mode==0) {
		for(i=0;i<N;i+=1) {
			j=7;
			k=3;
			a[i] = i*j*k;
		}
	} else if(mode==1) {
		j=7;
		k=3;
		for(i=0;i<N;i+=1) {
			a[i] = i*j*k;
		}
	} else {
	    printf("ERROR: INVALID MODE ENTERED\n");
	}
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);
    
    free(a);
}



Writing loopinvariantcode.c


In [14]:
!gcc -O0 -o loopinvariantcode loopinvariantcode.c


In [15]:
!./loopinvariantcode 0


USAGE: loopinvariantcode 0|1
  0=invariant in loop
  1=invariant outside loop
EXAMPLE: Loop invariant code inside or outside of loop
Running with mode 0
Elapsed Time = 0.451991


In [16]:
!./loopinvariantcode 1

USAGE: loopinvariantcode 0|1
  0=invariant in loop
  1=invariant outside loop
EXAMPLE: Loop invariant code inside or outside of loop
Running with mode 1
Elapsed Time = 0.457057


In [17]:
%%writefile strengthred.c
#include <stdio.h>
#include <math.h> //COMPILE WITH -lm OPTION!!!
#include <time.h>
#include <stdlib.h>

#define N 100000000

int main(int argc, char ** argv) {
    int i;
    double *a = (double *)malloc(sizeof(double)*N);
	int mode=-1;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: strengthred 0|1\n  0=pow(i,2)\n  1=i*i\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
        printf("EXAMPLE: strength reduction, x*x or pow(x,2)\n");
        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }

    start = clock();
	if(mode==0) {
		for(i=0;i<N;i+=1) {
			a[i] = pow((double)i,2.0);
		}
	} else if(mode==1) {
		for(i=0;i<N;i+=1) {
			a[i] = (double)i*(double)i;
		}
	} else {
		printf("ERROR: INVALID MODE ENTERED\n");
	}
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);

    free(a);    
}


Writing strengthred.c


In [18]:
!gcc -O0 -o strengthred strengthred.c -lm


In [19]:
!./strengthred 0


USAGE: strengthred 0|1
  0=pow(i,2)
  1=i*i
EXAMPLE: strength reduction, x*x or pow(x,2)
Running with mode 0
Elapsed Time = 1.451170


In [20]:
!./strengthred 1

USAGE: strengthred 0|1
  0=pow(i,2)
  1=i*i
EXAMPLE: strength reduction, x*x or pow(x,2)
Running with mode 1
Elapsed Time = 0.636081


In [21]:
%%writefile inlining.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>


#define macro_avg(x,y) ((x)+(y))/2
#define N 100000000

int func_avg(int x, int y) {
    return (x+y)/2;
}

int main(int argc, char ** argv) {
    int i;
    int * a = (int *)malloc(sizeof(int)*N);
	int mode=-1;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: inlining 0|1\n  0=use standard function\n  1=use macro\n");
    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
        printf("EXAMPLE: Inlining functions.");
        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }
    
	
    start = clock();
    if(mode==0) {
		for(i=0;i<N;i++) {
            a[i] = func_avg(i,1);
        }
	} else if(mode==1) {
		for(i=0;i<N;i++) {
            a[i] = macro_avg(i,1);
        }
	} else {
        printf("ERROR: INVALID MODE ENTERED\n");
    }
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);

    free(a);
}


Writing inlining.c


In [22]:
!gcc -O0 -o inlining inlining.c


In [23]:
!./inlining 0


USAGE: inlining 0|1
  0=use standard function
  1=use macro
EXAMPLE: Inlining functions.Running with mode 0
Elapsed Time = 0.522163


In [24]:
!./inlining 1

USAGE: inlining 0|1
  0=use standard function
  1=use macro
EXAMPLE: Inlining functions.Running with mode 1
Elapsed Time = 0.464151


In [25]:
%%writefile arrayfunc.c
#include <stdio.h>
#include <time.h>
#include <stdlib.h>

#define N 200000000

int func(int x) {
    return x*x;
}
void afunc(int *a) {
    int i;
    for(i=0;i<N;i++) a[i] = i*i;
}

int main(int argc, char ** argv) {
    int i;
    int * a = (int *)malloc(sizeof(int)*N);
    int mode=-1;
    clock_t start,end;
    double elapsed_time;

    printf("USAGE: arrayfunc 0|1\n  0=pass scalar to function within loop\n  1=pass array to function, function contains loop\n");
    fflush(stdout);

    if(argc>1) {
        sscanf(argv[1],"%d",&mode);
        printf("EXAMPLE: Passing arrays to functions or calling functions on elements of arrays.");
        printf("Running with mode %d\n",mode);
    } else {
        printf("Please provide a command line argument\n");
    }

    start = clock();
    if(mode==0) {
        for(i=0;i<N;i++) {
            a[i]=func(i);
        }
    } else if(mode==1) {
        afunc(a);
    } else {
        printf("ERROR: INVALID MODE ENTERED\n");
    }
    end = clock();
    elapsed_time = (end-start)/(double)CLOCKS_PER_SEC;
    printf("Elapsed Time = %lf\n",elapsed_time);

    free(a);
}


Writing arrayfunc.c


In [26]:
!gcc -O0 -o arrayfunc arrayfunc.c 


In [27]:
!./arrayfunc 0


USAGE: arrayfunc 0|1
  0=pass scalar to function within loop
  1=pass array to function, function contains loop
EXAMPLE: Passing arrays to functions or calling functions on elements of arrays.Running with mode 0
Elapsed Time = 0.991431


In [28]:
!./arrayfunc 1

USAGE: arrayfunc 0|1
  0=pass scalar to function within loop
  1=pass array to function, function contains loop
EXAMPLE: Passing arrays to functions or calling functions on elements of arrays.Running with mode 1
Elapsed Time = 0.978192
